In [141]:
import os
import pandas as pd
import h5py

%matplotlib inline

from IPython.display import display, HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [142]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [143]:
# Load data from daysim_outputs.h5, apply labels
daysim = h5py.File(r'../../../outputs/daysim/daysim_outputs.h5', 'r')

trip = pd.DataFrame()
for col in daysim['Trip'].keys():
    trip[col] = daysim['Trip'][col][:]
    
person = pd.DataFrame()
for col in daysim['Person'].keys():
    person[col] = daysim['Person'][col][:]
    
hh = pd.DataFrame()
for col in daysim['Household'].keys():
    hh[col] = daysim['Household'][col][:]
    
# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [144]:

special_taz = pd.read_csv(r'../../../scripts/summarize/inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [145]:
# Join RGC, low income, and minority data, based on home location

trip_hh = pd.merge(trip,hh,on='hhno',how='left')

trip_hh = pd.merge(trip_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [146]:
trip_hh = trip_hh[trip_hh['mode']!='School Bus']

## Mode Share
Based on home location

In [147]:
def mode_share(trip_hh):
    """
    Produce trip mode share for low income, minority, and regional populations
    """
    # Regional total
    df_reg = pd.DataFrame(
        trip_hh.groupby('mode').sum()['trexpfac']/trip_hh['trexpfac'].sum())
    df_reg.columns = ['Region']
    df_reg = df_reg.reset_index()
    
    # Low Income
    df = trip_hh[['trexpfac','Low Income','mode']].groupby(['Low Income','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Low Income').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Low Income', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_inc = df.pivot(index='Low Income', columns='mode', values='mode_share')
    df_inc.index = ['Other','People of Lower Income']
    
    # Minority
    df = trip_hh[['trexpfac','Minority','mode']].groupby(['Minority','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Minority').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Minority', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_race = df.pivot(index='Minority', columns='mode', values='mode_share')
    df_race.index = ['Other','People of Color']
    
    # Merge all dataframes
    df = pd.merge(pd.DataFrame(df_race.loc['People of Color']).reset_index(),
         pd.DataFrame(df_inc.loc['People of Lower Income']).reset_index())
    df = pd.merge(df_reg, df)
    df.index = df['mode']
    df = df.drop('mode', axis=1)
    return df

### All Trips

In [148]:
# pd.options.display.float_format = '{:.2f}'.format
df = mode_share(trip_hh)
df

,Region,People of Color,People of Lower Income
mode,,,
Bike,0.02,0.02,0.02
HOV2,0.23,0.23,0.23
HOV3+,0.16,0.17,0.17
SOV,0.40,0.38,0.38
Transit,0.03,0.04,0.03
Walk,0.16,0.16,0.17


### Commute Trips

In [149]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']
commute_trips = commute_trips[commute_trips['mode'] != 'School Bus']
df = mode_share(commute_trips)
df

,Region,People of Color,People of Lower Income
mode,,,
Bike,0.02,0.02,0.03
HOV2,0.10,0.10,0.10
HOV3+,0.04,0.05,0.04
SOV,0.66,0.64,0.64
Transit,0.06,0.07,0.06
Walk,0.11,0.12,0.13


### Access to Frequent Transit

In [150]:
df = pd.read_csv(r'../../../outputs/transit/freq_transit_access.csv')
# Join parcel data (transit access) to household recods
df = pd.merge(hh[['hhparcel','hhsize','hhtaz']],df[['PARCELID','dist_frequent']],how='left',left_on='hhparcel',right_on='PARCELID')

# join to geography fiels
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [151]:
# Only include househodls on parcels within 1/2 mile of frequent transit
max_dist = 0.5
df_freq = df[df['dist_frequent'] <= max_dist]

In [152]:
# Calculate percent of people within that range

In [153]:
df_lowinc = pd.DataFrame(df_freq.groupby('Low Income').sum()['hhsize']/df.groupby('Low Income').sum()['hhsize'])

In [154]:
df_minority = pd.DataFrame(df_freq.groupby('Minority').sum()['hhsize']/df.groupby('Minority').sum()['hhsize'])

In [155]:
reg_share = df_freq.sum()['hhsize']/df.sum()['hhsize']

In [156]:
pd.options.display.float_format = '{:.2f}'.format

In [157]:
_df = pd.DataFrame([reg_share,df_minority.loc[1].values[0],df_lowinc.loc[1].values[0]])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['Percent with Access to Frequent Transit Service']
_df['Percent with Access to Frequent Transit Service'] = _df['Percent with Access to Frequent Transit Service']*100
_df

,Percent with Access to Frequent Transit Service
Region,30.74
People of Color,40.38
People of Low Income,36.37


## Percent of People Walking and Biking for Transport

In [158]:
trip_person = pd.merge(trip,person,on=['hhno','pno'], how='left')
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
# bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk'])]
# Walk & bike modes, plus transit trips with walk access
bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk']) | ((trip_person['mode'] == 'Transit') & (trip_person['dorp'] > 0))]

df = bike_walk_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk'] = True

df = pd.merge(person_hh,df,on=['hhno','pno'], how='left')
df['bike_walk'] = df['bike_walk'].fillna(False)

In [159]:
pd.options.display.float_format = '{:,.1%}'.format
reg_active = pd.DataFrame(df.groupby('bike_walk').sum()['psexpfac']/df['psexpfac'].sum()).loc[True]['psexpfac']

In [160]:
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [161]:
_df = pd.DataFrame(df[df['Low Income'] == 1].groupby(['bike_walk']).count()).reset_index()
low_inc_active = _df[_df['bike_walk'] == True]['psexpfac'].tolist()[0]/_df.sum()['psexpfac']

In [162]:
_df = pd.DataFrame(df[df['Minority'] == 1].groupby(['bike_walk']).count()).reset_index()
minority_active = _df[_df['bike_walk'] == True]['psexpfac'].tolist()[0]/_df.sum()['psexpfac']

In [163]:
_df = pd.DataFrame([reg_active,minority_active,low_inc_active])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['% Walks or Bikes for Transportation']
_df

,% Walks or Bikes for Transportation
Region,30.8%
People of Color,32.2%
People of Low Income,32.9%


### Average Time Spent Walking

In [164]:
pd.options.display.float_format = "{0:.2f}".format
# walk_trips = trip_person[trip_person['mode'] == 'Walk']
# daily_walk_times = walk_trips.groupby(['hhno','pno']).sum()['travtime']

In [165]:
daily_tot_time = pd.DataFrame(trip_person.groupby(['pno','hhno','mode']).sum()['travtime'])
daily_tot_time = daily_tot_time.reset_index()
daily_walk_time = daily_tot_time[daily_tot_time['mode'] == 'Walk']

In [166]:
# Total walk time across region divided by total number of people
reg_walk_time = daily_walk_time['travtime'].sum()/person['psexpfac'].sum()

In [167]:
# Total for low inc and for rgc
df = pd.merge(daily_tot_time,hh[['hhno','hhtaz']],on='hhno', how='left')
df = pd.merge(df,rgc_taz,left_on='hhtaz',right_on='taz',how='left')
df = pd.merge(df,special_taz,left_on='hhtaz',right_on='TAZ',how='left')

In [168]:
low_inc_walk_time = df[df['Low Income'] == 1]['travtime'].sum()/person['psexpfac'].sum()
minority_walk_time = df[df['Minority'] == 1]['travtime'].sum()/person['psexpfac'].sum()

In [169]:
df = pd.DataFrame([reg_walk_time,minority_walk_time,low_inc_walk_time])
df.columns = ['Average Minutes Walking per Day']
df.index = ['Region','People of Color','People of Low Income']
df

,Average Minutes Walking per Day
Region,15.34
People of Color,30.82
People of Low Income,28.37


# Annual Out-of-Pocket Costs

In [170]:
annual_factor = 300
reg_costs = trip_hh[['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
low_inc_costs = trip_hh[trip_hh['Low Income'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
minority_costs = trip_hh[trip_hh['Minority'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor

In [171]:
df = pd.DataFrame([reg_costs,minority_costs,low_inc_costs])
df.columns = ['Annual Out-of-Pocket Costs']
df.index = ['Region','People of Color','People of Low Income']

# Out of pocket costs as percentage of household income
hh = pd.merge(hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
hh = pd.merge(hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

regional_avg_inc = hh.groupby('hhno').sum()['hhincome'].median()
low_inc_avg_inc = hh[hh['Low Income'] == 1].groupby('hhno').sum()['hhincome'].median()
minority_avg_inc = hh[hh['Minority'] == 1].groupby('hhno').sum()['hhincome'].median()

df_inc = pd.DataFrame([regional_avg_inc,low_inc_avg_inc,minority_avg_inc])
df_inc.columns = ['Median Income']
df_inc.index = ['Region','People of Low Income','People of Color']

df = pd.merge(df,df_inc,left_index=True, right_index=True)
df['% of Income'] = (df['Annual Out-of-Pocket Costs']/df['Median Income'])*100
df

,Annual Out-of-Pocket Costs,Median Income,% of Income
Region,3392.83,68610.00,4.95
People of Color,3114.21,59295.00,5.25
People of Low Income,2972.20,51041.00,5.82


In [172]:
# Percent of income for households making 30,000
df['% Income for $30000 income household'] = (df[['Annual Out-of-Pocket Costs']]/30000)*100
df['% Income for $20000 income household'] = (df[['Annual Out-of-Pocket Costs']]/20000)*100
df

,Annual Out-of-Pocket Costs,Median Income,% of Income,% Income for $30000 income household,% Income for $20000 income household
Region,3392.83,68610.00,4.95,11.31,16.96
People of Color,3114.21,59295.00,5.25,10.38,15.57
People of Low Income,2972.20,51041.00,5.82,9.91,14.86


### Daily VMT per capita

In [173]:
driver_trips = trip[(trip['dorp'] == 1) & (trip['mode'].isin(['SOV','HOV2','HOV3+']))]

In [174]:
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
person_hh = pd.merge(person_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')
person_hh = pd.merge(person_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(driver_trips,person_hh,on=['hhno','pno'], how='left')

In [175]:
reg_vmt = df['travdist'].sum()/person_hh['psexpfac'].sum()

In [176]:
vmt = pd.DataFrame(df[['Minority','travdist']].groupby('Minority').sum()['travdist'])
# Population totals
pop = person_hh[['Minority','psexpfac']].groupby(['Minority']).sum()
minority_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [177]:
vmt = pd.DataFrame(df[['Low Income','travdist']].groupby('Low Income').sum()['travdist'])
# Population totals
pop = person_hh[['Low Income','psexpfac']].groupby(['Low Income']).sum()
lowinc_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [ ]:
_df = pd.DataFrame([reg_vmt,minority_vmt,lowinc_vmt])
_df.columns = ['Average VMT per Capita']
_df.index = ['Region','People of Color','People of Lower Income']
_df

,Average VMT per Capita
Region,16.49
People of Color,14.68
People of Lower Income,14.38


# Average Auto Delay per Capita

In [ ]:
trip_auto = trip_hh[trip_hh['mode'].isin(['SOV','HOV2','HOV3+']) & (trip['dorp'] == 1)]
trip_auto['delay'] = trip_auto['travtime'] - trip_auto['sov_ff_time']/100.0

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
# Regional average
reg_delay = (trip_auto['delay'].sum()/person['psexpfac'].sum())*300/60

In [ ]:
low_inc_delay = trip_auto[trip_auto['Low Income'] == 1]['delay'].sum()/person_hh[person_hh['Low Income'] == 1][['psexpfac']].sum()
low_inc_delay = low_inc_delay.values[0]*300/60

In [ ]:
minority_delay = trip_auto[trip_auto['Minority'] == 1]['delay'].sum()/person_hh[person_hh['Minority'] == 1][['psexpfac']].sum()
minority_delay = minority_delay.values[0]*300/60

In [ ]:
df = pd.DataFrame([reg_delay,low_inc_delay,minority_delay])
df.columns = ['Average Annual Vehicle Delay (hours)']
df.index = ['Region','People of Low Income','People of Color']
df